# Supply Optimization
As a typical logistic problem, a grocer needs to minimize the number of suppliers while fully filling the inventory. More specifically, we have the following assumptions:

  1. there are $N$ different kinds of items $U=\{U_0,U_1,\cdots,U_{N-1}\}$
  2. there are $M$ different suppliers $S=\{S_0,S_1,\cdots,S_{M-1}\}$, and each of them covers a subset of our items, i.e., $S_i\subset U$.

This is in fact is a "Set Cover" problem, and D-Wave CQM solver is used to handle this problem:

## Import Libraries

In [1]:
from dimod import CQM, Binary, quicksum
from dwave.system import LeapHybridCQMSampler
import numpy as np

## Problem Set Up
Randomly generating items as a list $U$ and suppliers as a list of subsets $V$.

In [2]:
# -------------- Problem set up --------------
# items (e.g. [0,2,3,4,9])
U = list(set(np.random.randint(10, size=(10))))
# suppliers (e.g. [{0,2,3,4,9}, {0,9,2}, {0,2,3,4}, {0,9,2,3}, {0,2,3,4,9}])
S = [set(U[i] for i in np.random.randint(len(U), size=(8))) for j in range(5)]

# Print set up
print('------------- Problem set up -------------')
print('The universe is',U)
print('Number of elements in the universe: {:d}'.format(len(U)))

print('There are {:d} collections:'.format(len(S)))
for j in range(len(S)):
    print('Supplier{:d}:'.format(j), S[j])
print('Number of sets: N={:d}'.format(len(S)))

------------- Problem set up -------------
The universe is [0, 1, 2, 4, 5, 6, 7, 9]
Number of elements in the universe: 8
There are 5 collections:
Supplier0: {0, 1, 2, 6}
Supplier1: {0, 4, 5, 6, 7, 9}
Supplier2: {0, 2, 4, 5, 7, 9}
Supplier3: {1, 2, 4, 5, 7}
Supplier4: {2, 4, 6, 7, 9}
Number of sets: N=5


## Building CQM
Binary variables $y_j$ are used to indicate if supplier $S_j$ is chosen or not:

$$
y_j = 
\left\{
    \begin{array}{cl}
        1, & S_j\mathrm{\ \ is\ \ choosen}\\
        0, & \mathrm{otherwise}
    \end{array}
\right.
$$

In [3]:
# Build CQM
cqm = CQM()

# Create Binary variables
y = [Binary(j) for j in range(len(S))]

Our target is to minimize the total number of suppliers, that is, the sum of indicators $y_j$ should be minimized:

$$
\min \sum_{j=0}^{M-1} y_j.
$$

In [4]:
# -------------- Objective Function ------------------
# minimize total number of suppliers
# Add obj to CQM
cqm.set_objective(quicksum(y[j] for j in range(len(S))))

Meanwhile, all items should be covered by some suppliers. This constraint can be modelled by counting for every item the number of chosen suppliers which cover this item:

$$
\sum_{j=0}^{M-1} c_{ji}\cdot y_j \geq 1,\ \ \ \ i=0,\cdots,N-1,
$$
where $c_{ji}$ is an indicator computed from the given set ups for denoting wheather item $U_i$ is covered by supplier $S_j$:

$$
c_{ji} = 
\left\{
    \begin{array}{cl}
        1, & U_i \in S_j\\
        0, & \mathrm{otherwise}
    \end{array}
\right.
$$

In [5]:
# -------------- Constraints -----------------
# suppliers should cover all items
for i in range(len(U)):
    cqm.add_constraint( quicksum(int(U[i] in S[j])*y[j] for j in range(len(S))) >= 1,
                        label = 'cover item {:d}'.format(i) )


## Submit to CQM Solver

In [6]:
# -------------- Submit to CQM sampler ---------------
cqm_sampler = LeapHybridCQMSampler()
sampleset = cqm_sampler.sample_cqm(cqm, label = 'Supply Demo')

## Results

In [7]:
# -------------- Process the results ---------------
print('------------- Solution -------------')
feasible_sols = sampleset.filter(lambda row: row.is_feasible == True)
if not len(feasible_sols):
    print("\nNo feasible solution found.\n")
else:
    sol = [int(feasible_sols.first.sample[i]) for i in range(len(S))]
    print(sol)
    print('There are {:d} supplier selected.'.format(int(quicksum(sol))))
    suppliers = [f'suppiler{i}' for i in np.where(sol)[0]]
    print('Selected Suppliers:', suppliers)

------------- Solution -------------
[1, 0, 1, 0, 0]
There are 2 supplier selected.
Selected Suppliers: ['suppiler0', 'suppiler2']
